In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import spacy
from spacy.lang.en import English

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
from joblib import joblib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from ast import literal_eval

In [2]:
data_train = pd.read_csv("../../data/data_with_features/data_train_with_features.csv").drop(["Unnamed: 0"], axis=1)
data_test = pd.read_csv("../../data/data_with_features/data_test_with_features.csv").drop(["Unnamed: 0"], axis=1)

data_train = data_train.sample(frac=1).reset_index(drop=True)
data_test = data_test.sample(frac=1).reset_index(drop=True)

In [3]:
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(data_train["text"])

X_train, X_test, y_train, y_test = train_test_split(features, data_train["classification"], test_size=0.2, random_state=42)

model = svm.SVC(probability=True)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

  Scientific       1.00      0.98      0.99       101
        news       0.94      1.00      0.97        99
     reviews       0.97      0.97      0.97        93
       story       1.00      0.96      0.98       107

    accuracy                           0.98       400
   macro avg       0.98      0.98      0.98       400
weighted avg       0.98      0.98      0.98       400



In [12]:
new_text = data_test["text"][12]
new_text_features = vectorizer.transform([new_text])
probabilities = model.predict_proba(new_text_features)
true_class = data_test["classification"][12]
predicted_class = model.predict(new_text_features)

# Wahrscheinlichkeiten und vorhergesagte Klasse ausgeben
for i, probs in enumerate(probabilities):
    class_probabilities = ["{:.2f}%".format(prob * 100) for prob in probs]
    print("Klasse {}: {}".format(predicted_class, class_probabilities))
print("Vorhergesagte Klasse:", predicted_class, " | Wahre Klasse:", true_class)

Klasse ['Scientific']: ['100.00%', '0.00%', '0.00%', '0.00%']
Vorhergesagte Klasse: ['Scientific']  | Wahre Klasse: Scientific


In [27]:
data_train = pd.read_csv("../../data/data_with_features/data_train_with_features.csv").drop(["Unnamed: 0"], axis=1)
data_test = pd.read_csv("../../data/data_with_features/data_test_with_features.csv").drop(["Unnamed: 0"], axis=1)

data_train = data_train.sample(frac=1).reset_index(drop=True)
data_test = data_test.sample(frac=1).reset_index(drop=True)

In [28]:
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(data_train["text"])

X_train, X_test, y_train, y_test = train_test_split(features, data_train["classification"], test_size=0.2, random_state=42)

In [29]:
from keras.layers import LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers import Embedding

In [30]:
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(features.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
label_to_int = {label: i for i, label in enumerate(np.unique(data_train["classification"]))}
y_train = np.array([label_to_int[label] for label in y_train])
y_test = np.array([label_to_int[label] for label in y_test])

In [32]:
model.fit(X_train.toarray(), y_train, epochs=10, batch_size=16, verbose=1)

Epoch 1/10
100/100 [==============================] - 5s 44ms/step - loss: 0.7612 - accuracy: 0.8612
Epoch 2/10
100/100 [==============================] - 4s 43ms/step - loss: 0.0581 - accuracy: 0.9981
Epoch 3/10
100/100 [==============================] - 4s 43ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 4/10
100/100 [==============================] - 4s 39ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 5/10
100/100 [==============================] - 4s 43ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 6/10
100/100 [==============================] - 4s 42ms/step - loss: 7.6565e-04 - accuracy: 1.0000
Epoch 7/10
100/100 [==============================] - 4s 39ms/step - loss: 4.5966e-04 - accuracy: 1.0000
Epoch 8/10
100/100 [==============================] - 4s 40ms/step - loss: 2.5557e-04 - accuracy: 1.0000
Epoch 9/10
100/100 [==============================] - 4s 40ms/step - loss: 1.5063e-04 - accuracy: 1.0000
Epoch 10/10
100/100 [==============================] - 4s 40ms/step - loss:

In [33]:
loss, accuracy = model.evaluate(X_test.toarray(), y_test, verbose=1)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

13/13 [==============================] - 1s 36ms/step - loss: 0.0124 - accuracy: 0.9975
Test Loss: 0.012417732737958431
Test Accuracy: 0.9975000023841858


In [36]:
new_text = data_test["text"][0]
new_class = data_test["classification"][0]
new_text_features = vectorizer.transform([new_text])

predictions = model.predict(new_text_features.toarray())
predicted_class = np.argmax(predictions, axis=1)
predicted_probability = np.max(predictions, axis=1)

int_to_label = {i: label for label, i in label_to_int.items()}

predicted_labels = [int_to_label[prediction] for prediction in predicted_class]
for label, probability in zip(predicted_labels, predicted_probability):
    print(f"Vorhergesagte Klasse: {label}, Wahrscheinlichkeit: {probability}, Wahre Klasse: {new_class}.")

1/1 [==============================] - 0s 14ms/step
Vorhergesagte Klasse: news, Wahrscheinlichkeit: 0.9990744590759277, Wahre Klasse: news.


In [166]:
# mit Allem: 99% Scientific
# ohne Anhang & Vezeichnisse: 99% Scientific

In [37]:
model.save("../../models/classification/neuro_net_1.h5")